In [1]:
from allennlp_models.pretrained import load_predictor
import numpy as np
from  resit_utils import predict_and_transform_spanbert, evaluate_non_anaphoric_it, evaluate_number, evaluate_gender
import pandas as pd

In [2]:
#allennlp model (2018)
allen_bert = load_predictor('coref-spanbert')

lerc is not a registered model.
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Non-anaphoric it

In [3]:
non_anaphoric_path = '/Users/vicky/Desktop/Master/Period4/NLP/resit/data/non_anaphoric_it.csv'
non_anaphoric_df = pd.read_csv(non_anaphoric_path, delimiter=',')

In [4]:
predict_and_transform_spanbert(non_anaphoric_df, allen_bert)

/Users/vicky/Desktop/Master/Period4/NLP/resit/resit_utils.py:78: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df['predictions'] = np.array(predictions)


In [5]:
evaluate_non_anaphoric_it(non_anaphoric_df)

In [6]:
target_path = '/Users/vicky/Desktop/Master/Period4/NLP/resit/data/non_anaphoric_it_bert_preds.csv'
non_anaphoric_df.to_csv(target_path)

### NUMBER

In [3]:
num_path = '/Users/vicky/Desktop/Master/Period4/NLP/resit/data/number_position_synt_role.csv'
num_df = pd.read_csv(num_path, delimiter=',')
num_df.rename(columns={'sentence':'sentences'}, inplace=True)

In [4]:
predict_and_transform_spanbert(num_df, allen_bert)

/Users/vicky/Desktop/Master/Period4/NLP/resit/resit_utils.py:78: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df['predictions'] = np.array(predictions)


In [5]:
num_df

,Unnamed: 0,sentences,position,gold,syntactic_role,number,predictions
0,0,"After the car exploded, it was completely dest...",subordinate,"the car,it",subj,singular,"[[the car, it]]"
1,1,The car exploded. It was completely destroyed.,independent,"The car,It",subj,singular,"[[The car, It]]"
2,2,"After it exploded, the car was completely dest...",cataphora,"it,the car",subj,singular,[]
3,3,"After the car crashed, it was completely subme...",subordinate,"the car,it",subj,singular,"[[the car, it]]"
4,4,The car crashed. It was completely submerged.,independent,"The car,It",subj,singular,"[[The car, It]]"
...,...,...,...,...,...,...,...
1624,175,The protester stops the troupe. They leave.,independent,"the troupe,They",obj_subj,collective,"[[the troupe, They]]"
1625,176,"The protester stops them, and the troupe leave.",cataphora,"them,the troupe",obj_subj,collective,"[[them, the troupe]]"
1626,177,"The protester confronts the troupe, and they l...",subordinate,"the troupe,they",obj_subj,collective,"[[the troupe, they]]"
1627,178,The protester confronts the troupe. They leave.,independent,"the troupe,They",obj_subj,collective,"[[the troupe, They]]"


In [9]:
def evaluate_number(number_df):
    '''
    Evaluate the number challenge dataset based on the word clusters'''
    result=[]
    processed_gold = []
    for i, row in number_df.iterrows():
        prediction = row['predictions']
        gold = [row['gold']]
        print(gold)
        gold = gold[0].split(",")
        processed_gold.append(gold)

        if gold in prediction: #if the gold cluster is found inside the list of clusters
            score = 1
        else:
            score = 0
        
        result.append(score)
    
    number_df['results']=np.array(result)
    number_df.drop('gold', inplace=True, axis=1)
    number_df.insert(5, 'gold', processed_gold)

In [10]:
evaluate_number(num_df)

[['the car', 'it']]


AttributeError: 'list' object has no attribute 'split'

In [11]:
target_num_path = '/Users/vicky/Desktop/Master/Period4/NLP/resit/data/number_position_synt_role_bert_preds.csv'
num_df.to_csv(target_num_path)

### GENDER

In [12]:
gender_path = '/Users/vicky/Desktop/Master/Period4/NLP/resit/data/gender_anaphortype_ambiguity.csv'
gender_df = pd.read_csv(gender_path, delimiter = ",")

In [13]:
predict_and_transform_spanbert(gender_df, allen_bert, gender=True)

/Users/vicky/Desktop/Master/Period4/NLP/resit/resit_utils.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df[ref+'_predictions'] = np.array(predictions)
/Users/vicky/Desktop/Master/Period4/NLP/resit/resit_utils.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df[ref+'_predictions'] = np.array(predictions)


In [16]:
evaluate_gender(gender_df)

In [17]:
target_gender_path = '/Users/vicky/Desktop/Master/Period4/NLP/resit/data/gender_anaphortype_ambiguity_bert_preds.csv'
gender_df.to_csv(target_gender_path)